### 决策树


- 决策树是一种对实例进行分类的树状结构，在树中

In [4]:
import numpy as np
# 初分类 对单独列进行元素和label的分类

x = np.array(([[1,2,3,4],[2,3,4,5],[3,4,5,5],[3,4,2,3],[3,4,1,1],[1,5,2,2],[5,4,1,1],[1,4,2,3],[3,2,1,1],[3,4,5,1],[2,4,1,3],[1,1,2,2],[2,4,2,2]]))
y = np.array(([1,0,0,1,1,1,1,1,0,0,0,0,0]))
np.column_stack((y, x))

array([[1, 1, 2, 3, 4],
       [0, 2, 3, 4, 5],
       [0, 3, 4, 5, 5],
       [1, 3, 4, 2, 3],
       [1, 3, 4, 1, 1],
       [1, 1, 5, 2, 2],
       [1, 5, 4, 1, 1],
       [1, 1, 4, 2, 3],
       [0, 3, 2, 1, 1],
       [0, 3, 4, 5, 1],
       [0, 2, 4, 1, 3],
       [0, 1, 1, 2, 2],
       [0, 2, 4, 2, 2]])

In [5]:
# 对第column列进行数据划分
# 返回一个字典，字典{factor:[{label_1,count},{lable_2, count}]
def divide_data(x, y, column):
    data = np.column_stack((y, x)) # 对数据进行结合，方便遍历数据
    dict_factor_lable_count = {}
    dict_index = {}
    count = 0
    for row_x_y in data:
        factor_key = dict_factor_lable_count.keys()
        factor = row_x_y[1+column] # 第column列元素
        lable = row_x_y[0]         # lable
        if( factor in factor_key):
            dict_index[factor].append({count})
            count += 1
            lable_count_array = dict_factor_lable_count[factor]
            for index in range(0,len(lable_count_array)):
                if lable in lable_count_array[index].keys():
                    lable_count_array[index][lable] = lable_count_array[index][lable] + 1
                    break
                elif index + 1 == len(lable_count_array):
                    tmp_dict = {lable: 1}
                    lable_count_array.append(tmp_dict)
                    break
        else:
            dict_factor_lable_count[factor] = [{lable: 1}]
            dict_index[factor] = [{count}]
            count +=1
    return (dict_factor_lable_count, dict_index) 
    

# ({1: [{1: 3}, {0: 1}], 2: [{0: 3}], 3: [{0: 3}, {1: 2}], 5: [{1: 1}]},
#  {1: [{0}, {5}, {7}, {11}], 2: [{1}, {10}, {12}], 3: [{2}, {3}, {4}, {8}, {9}], 5: [{6}]})
#  对x 的第0列进行划分，可以得到，当第0列元素是1时，lable=1的样本有3个，lable=0的样本有1个，依此类推。
#  数据按着第column列进行划分，得到数据的下标。
print(divide_data(x,y,0))

({1: [{1: 3}, {0: 1}], 2: [{0: 3}], 3: [{0: 3}, {1: 2}], 5: [{1: 1}]}, {1: [{0}, {5}, {7}, {11}], 2: [{1}, {10}, {12}], 3: [{2}, {3}, {4}, {8}, {9}], 5: [{6}]})


In [6]:
def get_lable_count(y):
    all_lable_count = {}
    for lable in y:
        if lable in all_lable_count.keys():
            all_lable_count[lable] = all_lable_count[lable] + 1
        else:
            all_lable_count[lable] = 1
    return all_lable_count

$Ent(D^v) = - \sum_\limits{i=1}^{n}p_i log p_i$

In [16]:
(dict_factor_lable_count, dict_index) = divide_data(x,y,0)
# 计算信息熵
import math
def calcu_gain(dict_factor_lable_count, dict_index):
    factor_set = dict_factor_lable_count.keys()
    list_entropy = []
    count = 0
    for factor in factor_set:
        lable_count_array=dict_factor_lable_count[factor]
        factor_all_count = len(dict_index[factor])
        sum = 0
        for lable_count in lable_count_array:
            for lable in lable_count.keys():
                rate = lable_count[lable] / factor_all_count
                sum += rate*math.log(rate,2)
        list_entropy.append(-sum)

    return list_entropy
        
print(calcu_gain(dict_factor_lable_count, dict_index))

[0.8112781244591328, -0.0, 0.9709505944546686, -0.0]


In [20]:
class Node:
    next_factor_order = -1
    list_node = []
    list_factors = []
    is_lable = False
    lable = None
    
    def __init__(self, next_factor_order, list_node, list_factors, is_lable, lable):
        self.next_factor_order = next_factor_order
        self.list_node = list_node
        self.list_factors = list_factors
        self.is_lable = is_lable
        if is_lable:
            self.lable = lable
    pass
    